# SQL - 데이터 전체 조회

In [1]:
import sqlite3

conn = sqlite3.connect('member.db')
cursor = conn.cursor()

#데이터조회 SQL문 실행
cursor.execute("select * from meminfo")
rows = cursor.fetchall()#모든 레코드 가져오기, 테이블 : stone, flower

for row in rows:
    print(row)
    
conn.close()

('wynam', 'n1234', '남운영', '2019-08-20 13:40:50')
('jklee', 'l3456', '이전경', '2019-08-20 14:50:10')


# SQL - 열 이름으로 조회

In [2]:
import sqlite3
conn = sqlite3.connect('member.db')

conn.row_factory = sqlite3.Row
cursor = conn.cursor()
cursor.execute('select * from meminfo')#meminfo 테이블 내 모든 데이터
#rows = cursor.fetchall()#rows에 레코드 저장

for row in cursor.fetchall():
    print("ID : %s, PASSWD : %s, name : %s, time : %s" %
          (row['id'], row['passwd'], row['name'], row['time']))
    
for row in rows:
    print(row)

conn.close()

ID : wynam, PASSWD : n1234, name : 남운영, time : 2019-08-20 13:40:50
ID : jklee, PASSWD : l3456, name : 이전경, time : 2019-08-20 14:50:10
('wynam', 'n1234', '남운영', '2019-08-20 13:40:50')
('jklee', 'l3456', '이전경', '2019-08-20 14:50:10')


# 기상청 데이터 응용

In [10]:
import urllib.request
from bs4 import BeautifulSoup
#기상청 날씨페이지 로드
page = urllib.request.urlopen('http://www.weather.go.kr/weather/\
observation/currentweather.jsp').read()
soup = BeautifulSoup(page, "html.parser")

#DB 연결
import sqlite3
conn = sqlite3.connect('Weather.db')
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

In [8]:
#현재시간 태그 검색
current_time = soup.find('input',{'class':'time'})
#날짜 시간 분리 후 SQLite3 형식의 날짜 시간 문자열로 변환
dt = current_time['value'].split('.')
year, month, date, time = dt[0], dt[1], dt[2], dt[3]
date_time = year + "-" + month + "-" + date + " " + time
print(date_time)

2021-10-07 05:00


In [9]:
#현재 날씨테이블 검색
table = soup.find('table', {'class':'table_develop3'})
data = []

for tr in table.find_all('tr'):
    tds = tr.find_all('td')
    for td in tds:
        if td.find('a'):
            city = td.find('a').text
            temperature = tds[5].text
            humidity = tds[9].text
            data.append([city, temperature, humidity])
            #레코드 삽입 SQL문 실행
            cursor.execute('''
            insert into CityWeather (city, temperature, humidity, dtime) values (?,?,?,?)''',
                           (city, temperature, humidity, date_time))
            
conn.commit()#데이터베이스 내보내기
#데이터 조회 SQL문 실행, 31로 검색 안되서 20로 변경
cursor.execute("select * from CityWeather where temperature > 5")
rows = cursor.fetchall()

for row in rows:
    print("도시 : %s, 온도 : %s, 습도 : %s, 시간 : %s"%
         (row['city'],row['temperature'], row['humidity'], row['dtime']))
    
conn.close()

도시 : 울산, 온도 : 18.1, 습도 : 98, 시간 : 2021-10-07 05:00


In [26]:
cursor.execute("select  max(temperature) as 최고기온, city as 최고도시, a.최저기온, a.최저도시 from CityWeather,\
 (select min(temperature) as 최저기온, city as 최저도시 from CityWeather) a ")
rows = cursor.fetchall()

for row in rows:
    print("최저기온 도시 : %s, 최저기온 : %s, 최고기온 도시 : %s, 최고기온 : %s"%
         (row['최저도시'],row['최저기온'],row['최고도시'],row['최고기온']))

최저기온 도시 : 대관령, 최저기온 : 11.6, 최고기온 도시 : 서귀포, 최고기온 : 22.5


# 임의의 프로그램

- 이전의 생일 관련 미신 데이터 출력
- 탄생석, 탄생화 DB화하여 출력

In [27]:
import urllib.request
from bs4 import BeautifulSoup
import sqlite3

conn = sqlite3.connect('AllOfMyth.db')#DB파일 연결
conn.row_factory = sqlite3.Row
cursor = conn.cursor()

#위키피디아 탄생석 사이트
bs_page = urllib.request.urlopen("https://ko.wikipedia.org/wiki/\
%ED%83%84%EC%83%9D%EC%84%9D").read()
bs_soup = BeautifulSoup(bs_page, "html.parser")

#탄생석 정보 테이블 검색
bs_table = bs_soup.find("table", {"class":"wikitable"})

#탄생석 데이터 저장
bs_data = []

#테이블 내의 데이터를 가져옴
for tr in bs_table.find_all("tr"):
    tds = tr.find_all("td")
    for td in tds:
        bs_data.append(td.text)
        
#수집한 데이터 DB에 삽입
temp = 0
while temp < len(bs_data):
    cursor.execute('''
            insert into stone(date, name, mean) values (?,?,?)''',
                           (bs_data[temp], bs_data[temp+1], bs_data[temp+2]))
    temp = temp + 3

#커밋    
conn.commit()

In [28]:
#탄생화 정보 사이트
bf_page = urllib.request.urlopen("https://m.blog.naver.com/\
PostView.naver?isHttpsRedirect=true&blogId=ufo5405&logNo=221652151845").read()
bf_soup = BeautifulSoup(bf_page, "html.parser")

#탄생화 정보 검색
bf_center = bf_soup.find_all("p", {"align":"center"})
#탄생화 데이터 저장
bf_date = []
bf_name = []
bf_mean = []

#탄생화 데이터 추출
for bf in bf_center:
    day_split = bf.text.split(":")
    if len(day_split) >= 2:
        bf_date.append(day_split[0])
        name_split = day_split[1].split(" - ")
        bf_name.append(name_split[0])
        bf_mean.append(name_split[1])

#DB에 데이터 삽입
temp = 0
while temp < len(bf_date):
    cursor.execute('''
            insert into flower (date, name, mean) values (?,?,?)''',
                           (bf_date[temp], bf_name[temp], bf_mean[temp]))
    temp = temp + 1
    
#저장후 연결종료
conn.commit()
conn.close()

In [29]:
from flask import Flask, render_template, request
import urllib.request
from bs4 import BeautifulSoup
import sqlite3

app = Flask(__name__)
#시작페이지 : 생년월일 입력
@app.route("/main")
def join_main():
    return render_template("main.html")

#결과출력페이지 : 입력된 정보로 검색한 내용 나열
@app.route("/show", methods=["POST"])
def show_myth() -> 'html':
    #오늘의 운세 페이지
    today_page = urllib.request.urlopen("https://unse.daily.co.kr/?p=zodiac").read()
    today_soup = BeautifulSoup(today_page, "html.parser")
    
    #년월일 가져오기, 스플릿
    ymd = request.form['ymd']
    ymd_split = ymd.split('-')
    
    #띠 계산
    animals = ['쥐','소','호랑이','토끼','용','뱀','말','양','원숭이','닭','개','돼지']
    zodiac = animals[(int(ymd_split[0]) - 4) % 12]
    z_code = "code_" + str(animals.index(zodiac) + 1)
    z_year = ymd_split[0][2:]
    
    #웹페이지 운세목록 추출
    fortune = today_soup.find("ul", {"id":z_code})

    #띠 공통 운세
    total_text = fortune.find("p").text

    #띠 연도에 맞는 운세
    year_text = ""
    for i in fortune.find_all("li"):
        if i.find("span").text == str(z_year) + "년생":
            year_text = i.find("p").text
            
    #탄생화와 탄생석 DB에 연결
    conn = sqlite3.connect('AllOfMyth.db')
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    
    #월과 일 입력시 0이 앞에 있는 경우 필터링 (01월 -> 1월)
    clear_month = ymd_split[1].replace("0","") if ymd_split[1][0] == "0" else ymd_split[1]
    clear_day = ymd_split[2].replace("0","") if ymd_split[2][0] == "0" else ymd_split[2]
    
    #탄생화와 탄생석에 대한 데이터 추출
    f_date = clear_month + "월 " + clear_day + "일"
    flower = cursor.execute("select * from flower where date like \"%s%%\"" % (f_date)).fetchone()
    stone = cursor.execute("select * from stone where date like \"%s%%\"" % (clear_month)).fetchone()
    
    conn.close()
    
    return render_template("show.html", ymd_split = ymd_split, zodiac = zodiac, total_text = total_text,
                          year_text = year_text, flower = flower, stone = stone)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Oct/2021 05:39:40] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [07/Oct/2021 05:39:44] "GET /main HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 05:39:52] "POST /show HTTP/1.1" 200 -


In [ ]:
from flask import Flask, render_template, request
import urllib.request
from bs4 import BeautifulSoup
import sqlite3

app = Flask(__name__)
#시작페이지 : 생년월일 입력
@app.route("/main")
def join_main():
    return render_template("main.html")

#결과출력페이지 : 입력된 정보로 검색한 내용 나열
@app.route("/show", methods=["POST"])
def show_myth() -> 'html':
    #오늘의 운세 페이지
    today_page = urllib.request.urlopen("https://unse.daily.co.kr/?p=zodiac").read()
    today_soup = BeautifulSoup(today_page, "html.parser")
    
    #년월일 가져오기, 스플릿
    ymd = request.form['ymd']
    ymd_split = ymd.split('-')
    
    #띠 계산
    animals = ['쥐','소','호랑이','토끼','용','뱀','말','양','원숭이','닭','개','돼지']
    zodiac = animals[(int(ymd_split[0]) - 4) % 12]
    z_code = "code_" + str(animals.index(zodiac) + 1)
    z_year = ymd_split[0][2:]
    
    #웹페이지 운세목록 추출
    fortune = today_soup.find("ul", {"id":z_code})

    #띠 공통 운세
    total_text = fortune.find("p").text

    #띠 연도에 맞는 운세
    year_text = ""
    for i in fortune.find_all("li"):
        if i.find("span").text == str(z_year) + "년생":
            year_text = i.find("p").text
            
    #탄생화와 탄생석 DB에 연결
    conn = sqlite3.connect('AllOfMyth.db')
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    
    #월과 일 입력시 0이 앞에 있는 경우 필터링 (01월 -> 1월)
    clear_month = ymd_split[1].replace("0","") if ymd_split[1][0] == "0" else ymd_split[1]
    clear_day = ymd_split[2].replace("0","") if ymd_split[2][0] == "0" else ymd_split[2]
    
    #탄생화와 탄생석에 대한 데이터 추출
    f_date = clear_month + "월 " + clear_day + "일"
    flower = cursor.execute("select * from flower where date like \"%s%%\"" % (f_date)).fetchone()
    stone = cursor.execute("select * from stone where date like \"%s%%\"" % (clear_month)).fetchone()
    
    conn.close()
    
    return render_template("show.html", ymd_split = ymd_split, zodiac = zodiac, total_text = total_text,
                          year_text = year_text, flower = flower, stone = stone)

if __name__ == "__main__":
    app.run()